In [1]:
import pandas as pd
import numpy as np
import os 
import string
import sys
import time
start = time.time()

# ETL

***

In [2]:
# Set File Dir for Weekly Data
file_dir = 'C:\\MSCA\\2020.04 31009 Machine Learning and Predictive Analytics\\Final Project\\test\\'

# Column Info
p_info = ['x', 'y', 's', 'a', 'dis', 'o', 'dir']
routes = ['ANGLE', 'CORNER', 'CROSS', 'FLAT', 'GO','HITCH', 'IN', 'OUT','POST', 'SCREEN', 'SLANT', 'WHEEL']
p_list = p_info+routes

# Import Player Data
players = pd.read_csv('players.csv')
players = players.append({'nflId':0,
                          'height':0,
                          'weight':0,
                          'birthDate': '2018-01-01',
                          'collegeName':'None',
                          'position':'BALL', 'displayName':'BALL'}, ignore_index=True)
players['nflId'] = players['nflId'].astype(np.int64)

# Import Plays data
plays = pd.read_csv('plays.csv')

# Create Function for Filling in Data
def nfl_norm(c):
    for i in pf.index:
        e_id = pf['e_id'][i]
        c_x = c+'_'+str(pf['nflId'][i])
        v_x =  pf[c][i]
        n_i = pf_norm.index[pf_norm['e_id'] == e_id]
        pf_norm.at[n_i, c_x] = v_x

In [3]:
for filename in os.listdir(file_dir):
    path = file_dir+filename
    pf = pd.read_csv(path)
    
    # Create e_id field and gp_id
    pf['e_id'] = pf['gameId'].map(str)+pf['playId'].map(str)+pf['frameId'].map(str)
    pf['e_id'] = pf['e_id'].astype(np.int64)
    
    pf['gp_id'] = pf['gameId'].map(str)+pf['playId'].map(str)
    pf['gp_id'] = pf['gp_id'].astype(np.int64)
    
    # Change nflId to int
    pf = pf.fillna(0)
    pf['nflId'] = pf['nflId'].astype(np.int64)
    
    # Create Play Result Field
    passing_df = pf[pf['event'].isin(['pass_outcome_touchdown',
                                      'pass_outcome_caught',
                                      'pass_outcome_incomplete',
                                      'pass_outcome_interception'])]
    
    passing_df = passing_df[['gp_id', 'event']]
    passing_df.columns = ['gp_id', 'result']  
    passing_df = passing_df.drop_duplicates()
    pf = pd.merge(pf, passing_df['result'], left_on=pf['gp_id'], right_on=passing_df['gp_id'], how='left')
    
    ########  ADD FEATURES HERE  ########
    
    # Isolate Pass Forward
    pf = pf[pf['event'] == 'pass_forward']
    
    # Create Base Dummy Fields for Route
    pf['route'] = pf['route'].replace(['undefined'], 0)
    r = pd.get_dummies(pf['route'], columns=routes, drop_first=True, dtype=int)
    pf = pd.concat([pf,r], axis = 1)
    
    # Crate Normalized Template
    pf_norm = pf[['e_id', 'gp_id', 'result']]
    pf_norm = pf_norm.drop_duplicates()
    pf_norm = pf_norm.reset_index(drop=True)
    
    for i in players['nflId']:
        i =str(i)
        for j in p_list:
            pf_norm[j+'_'+i] = 0
            
    pf_norm = pf_norm.fillna(0)
    
    for p in p_list:
        nfl_norm(p)
    
    # Export to CSV
    nf = 'norm_'+filename
    pf_norm.to_csv(nf)

In [4]:
end = time.time()
print(end-start)

976.1059892177582


In [5]:
p_col = list(pf_norm.columns.values)
print(p_col[:5])

['e_id', 'gp_id', 'result', 'x_2539334', 'y_2539334']
